<!-- PROFILE LINK -->

<h1 style = "text-align:center; font-size: 30px">CHAT USING SOCKETS</h1>
<a href ="https://github.com/moriones2001">
<img src="https://avatars.githubusercontent.com/u/100123414?v=4" style="float:left;vertical-align:centre" width="110" height="110" title = "Iván García Moriones 21907762">
<a href ="https://github.com/moriones2001?tab=repositories">
<img src="https://media-exp1.licdn.com/dms/image/C4D0BAQEu_Aa76fcmPw/company-logo_200_200/0/1630998679429?e=2159024400&v=beta&t=2feC9GG7RWCWizTqsK5HzDWB1TL2DOQuAVwnMf6FFvk" title = "M41" alt="UE" width="110" height="110" style="float:right;vertical-align:centre">

In [ ]:
import socket
import threading
import sys
import pickle
import os

#Existen metodos deprecated, así que con esta libreria los ocultamos de la consola
import warnings
warnings.filterwarnings("ignore")

class Servidor():

	def __init__(self, host=socket.gethostname(), port=int(input("Que puerto quiere usar ? Deben ser los primeros 5 dígitos del expediente "))):
		#Lista de conexiones y nicknames que se conectan
		self.clientes = []
		self.nicknames = []
		print('\nSu IP actual es : ',socket.gethostbyname(host))
		print('\n\tProceso con PID = ',os.getpid(), '\n\tHilo PRINCIPAL con ID =',threading.currentThread().getName(), '\n\tHilo en modo DAEMON = ', threading.currentThread().isDaemon(), '\n\tTotal Hilos activos en este punto del programa =', threading.active_count())
		self.s = socket.socket()
		self.s.bind((str(host), int(port)))
		self.s.listen(30)
		self.s.setblocking(False)
		threading.Thread(target=self.aceptarC, daemon=True).start()
		threading.Thread(target=self.procesarC, daemon=True).start()

		while True:
			msg = input('\n << SALIR = 1 >> \n')
			if msg == '1':
				print(" **** Me piro vampiro; cierro socket y mato SERVER con PID = ", os.getpid())
				self.s.close()
				sys.exit()
			else: pass

	def aceptarC(self):
		print('\nHilo ACEPTAR con ID =',threading.currentThread().getName(), '\n\tHilo en modo DAEMON = ', threading.currentThread().isDaemon(),'\n\tPertenece al PROCESO con PID', os.getpid(), "\n\tHilos activos TOTALES ", threading.active_count())
		while True:
			try:
				conn, addr = self.s.accept()
				print(f"\nConexion aceptada via {addr}")
				conn.setblocking(False)
				self.clientes.append(conn)
				#Primer mensaje: Recuperamos NICK del cliente
				data = conn.recv(32).decode()
				if data: 
					nickname = data
					print("NICK conectado: ", nickname)
					self.nicknames.append(nickname)

			except: pass

	def procesarC(self):
		print('\nHilo PROCESAR con ID =',threading.currentThread().getName(), '\n\tHilo en modo DAEMON = ', threading.currentThread().isDaemon(),'\n\tPertenece al PROCESO con PID', os.getpid(), "\n\tHilos activos TOTALES ", threading.active_count())
		while True:
			if len(self.clientes) > 0:
				for c in self.clientes:
					try:
						data = c.recv(32)
						if data: self.broadcast(data,c)
						with  open('u21907762AI1.txt','a') as f:
							f.write(pickle.loads(data) + '\n')

					except: pass

	def broadcast(self, msg, cliente):
		for c in self.clientes:
			print("Clientes conectados Right now = ", len(self.clientes),' ',self.nicknames,'\tMENSAJE DE: ',pickle.loads(msg))
			try:
				if c != cliente: 
					#print(pickle.loads(msg))
					c.send(msg)

			except: self.clientes.remove(c)
		
arrancar = Servidor()

In [ ]:
import threading
import sys
import socket
import pickle
import os

#Existen metodos deprecated, así que con esta libreria los ocultamos de la consola

import warnings
warnings.filterwarnings("ignore")

class Cliente():
	#nickname=""
	def __init__(self, host=input("Introduzca la IP del servidor :  "),port=int(input("Introduzca el PUERTO del servidor,tus primeros 5 dígitos del expediente: ")),nickname=input("Introduzca un nickname, que será tu correo de la universidad: ")):
		self.s = socket.socket()
		self.s.connect((host, int(port)))
		#self.s.connect(('192.168.1.62', int('666')))  #Conexion rapida para pruebas
		
		#Primer mensaje: Enviamos NICK al servidor
		self.nickname=nickname
		self.s.send(nickname.encode())

		print('\n\tProceso con PID = ',os.getpid(), '\n\tHilo PRINCIPAL con ID =',threading.currentThread().getName(), '\n\tHilo en modo DAEMON = ', threading.currentThread().isDaemon(),'\n\tTotal Hilos activos en este punto del programa =', threading.active_count())
		threading.Thread(target=self.recibir, daemon=True).start()
		
		while True:
			#msg = input('\nEscriba texto ?   ** Enviar = ENTER   ** Salir Chat = 1: ')
			msg = input('\nNiCKNAME: ' + self.nickname + '\tEscriba texto ?   ** Enviar = ENTER   ** Salir Chat = 1\n')
			if msg != '1' : self.enviar(msg)
			else:
				print(" **** Me piro vampiro; cierro socket y mato al CLIENTE con PID = ", os.getpid())
				self.s.close()
				sys.exit()

	def recibir(self):
		print('\nHilo RECIBIR con ID =',threading.currentThread().getName(), '\n\tPertenece al PROCESO con PID', os.getpid(), "\n\tHilos activos TOTALES ", threading.active_count())
		while True:
			try:
				data = self.s.recv(32)
				if data: print(f'PARA:{self.nickname} \tDESDE: {pickle.loads(data)}')

			except: pass

	def enviar(self, msg):
		mensaje = f"{self.nickname}: {msg}"
		self.s.send(pickle.dumps(mensaje))
		#self.s.send(pickle.dumps(msg))

arrancar = Cliente()